# Chapter 9: Pole Placement Controllers

## Example 9.10: Pole_placement_controller_IBM_Lotus_Domino_server.sce

In [ ]:
// Pole placement controller IBM Lotus Domino server, discussed in Example 9.9 on page 341.
// 9.10  

exec('desired.sci',-1);
exec('pp_im.sci',-1);
exec('zpowk.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polsplit3.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('polmul.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polyno.sci',-1);

// Control of IBM lotus domino server
// Transfer function
B = 0.47; A = [1 -0.43]; k = 1; 
[zk,dzk] = zpowk(k);
 
// Transient specifications
rise = 10; epsilon = 0.01; Ts = 1;
phi = desired(Ts,rise,epsilon);

// Controller design
Delta = [1 -1];  // internal model of step used
[Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,Delta);

// Simulation parameters for stb_disc.xcos
st = 1; // desired change
t_init = 0; // simulation start time
t_final = 40; // simulation end time
C = 0; D = 1; N_var = 0;

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Rcp1,Rcp2] = cosfil_ip(1,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Bp,Ap] = cosfil_ip(B,A); // B/A
[zkp1,zkp2] = cosfil_ip(zk,1); // zk/1
[Cp,Dp] = cosfil_ip(C,D); // C/D




## Example 9.11: Pole_placement_controller_for_motor_problem.sce

In [ ]:
// Pole placement controller for motor problem, discussed in Example 9.10 on page 343.
// 9.11

exec('desired.sci',-1);
exec('pp_im.sci',-1);
exec('myc2d.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polsplit3.sci',-1);
exec('zpowk.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polyno.sci',-1);

// Motor control problem
// Transfer function
a1 = [-1 0; 1 0]; b1 = [1; 0]; c1 = [0 1]; d1 = 0;
G = syslin('c',a1,b1,c1,d1); Ts = 0.25;
[B,A,k] = myc2d(G,Ts);

// Transient specifications
rise = 3; epsilon = 0.05; 
phi = desired(Ts,rise,epsilon);

// Controller design
Delta = 1; // No internal model of step used
[Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,Delta);

// simulation parameters for c_ss_cl.xcos
st = 1;  //desired change in position
t_init = 0;  //simulation start time
t_final = 10;  //simulation end time
xInitial = [0 0];  //initial conditions
N = 1; C = 0; D = 1; N_var = 0;

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Np,Rcp] = cosfil_ip(N,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Cp,Dp] = cosfil_ip(C,D); // C/D




## Example 9.12: Procedure_to_split_a_polynomial_into_good_and_bad_factors.sci

In [ ]:
// Procedure to split a polynomial into good and bad factors, as discussed in Sec. 9.5. The factors that have roots outside unit circle or with negative real parts are defined as bad.
// 9.12

// function [goodpoly,badpoly] = polsplit3(fac,a)
// Splits a scalar polynomial of z^{-1} into good and bad 
// factors. Input is a polynomial in increasing degree of 
// z^{-1}.  Optional input is a, where a <= 1.
// Factors that have roots outside a circle of radius a or
// with negative roots will be called bad and the rest
// good. If a is not specified, it will be assumed as 1.

function [goodpoly,badpoly] = polsplit3(fac,a)
if argn(2) == 1, a = 1; end  
if a>1 error('good polynomial also is unstable'); end
fac1 = poly(fac(length(fac):-1:1),'z','coeff');
rts = roots(fac1);
rts = rts(length(rts):-1:1);

// extract good and bad roots
badindex = mtlb_find((abs(rts)>=a-1.0e-5)|(real(rts)<-0.05));
badpoly = coeff(poly(rts(badindex),'z'));
goodindex = mtlb_find((abs(rts)<a-1.0e-5)&(real(rts)>=-0.05));
goodpoly = coeff(poly(rts(goodindex),'z'));

// scale by equating the largest terms
[m,index] = max(abs(fac));
goodbad = convol(goodpoly,badpoly);
goodbad = goodbad(length(goodbad):-1:1);
factor1 = fac(index)/goodbad(index);
goodpoly = goodpoly * factor1;
goodpoly = goodpoly(length(goodpoly):-1:1);
badpoly = badpoly(length(badpoly):-1:1);
endfunction;

## Example 9.13: Pole_placement_controller_without_intra_sample_oscillations.sci

In [ ]:
// Pole placement controller without intra sample oscillations, as discussed in Sec. 9.5.
// 9.13

// function [Rc,Sc,Tc,gamma,phit] = pp_im2(B,A,k,phi,Delta,a)
// 2-DOF PP controller with internal model of Delta and without
// hidden oscillations

function [Rc,Sc,Tc,gamm,phit] = pp_im2(B,A,k,phi,Delta,a)

if argn(2) == 5, a = 1; end
dphi = length(phi)-1;

// Setting up and solving Aryabhatta identity
[Ag,Ab] = polsplit3(A,a); dAb = length(Ab) - 1;
[Bg,Bb] = polsplit3(B,a); dBb = length(Bb) - 1;

[zk,dzk] = zpowk(k);

[N,dN] = polmul(Bb,dBb,zk,dzk);
dDelta = length(Delta)-1;
[D,dD] = polmul(Ab,dAb,Delta,dDelta);

[S1,dS1,R1,dR1] = xdync(N,dN,D,dD,phi,dphi);

// Determination of control law
Rc = convol(Bg,convol(R1,Delta)); Sc = convol(Ag,S1);
Tc = Ag; gamm = sum(phi)/sum(Bb); 

// Total characteristic polynomial
phit = convol(phi,convol(Ag,Bg));
endfunction;

## Example 9.14: Controller_design.sce

In [ ]:
// Controller design for the case study presented in Example 9.12 on page 347. 
// 9.14

exec('tf.sci',-1);
exec('desired.sci',-1);
exec('zpowk.sci',-1);
exec('myc2d.sci',-1);
exec('polsplit3.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('pp_im2.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('cosfil_ip.sci',-1);

num = 200;
den = convol([0.05 1],[0.05 1]);
den = convol([10 1],den);
G = tf(num,den); Ts = 0.025; 
num = G('num'); den = G('den');
// iodel = 0;
[B,A,k] = myc2d(G,Ts);
[zk,dzk] = zpowk(k); //int1 = 0;

// Transient specifications
a = 0.9; rise = 0.24; epsilon = 0.05;
phi = desired(Ts,rise,epsilon);

// Controller design
Delta = [1 -1]; // internal model of step is present
[Rc,Sc,Tc,gamm] = pp_im2(B,A,k,phi,Delta,a);

// margin calculation
Lnum = convol(Sc,convol(B,zk));
Lden = convol(Rc,A);
L = tf(Lnum,Lden,Ts);
Gm = g_margin(L); //---- Does not match --------------- (in dB)
Pm = p_margin(L); //---- Convergence problem --------------- (in degree)

num1 = 100; den1 = [10 1];
Gd = tf(num1,den1);  //-------
[C,D,k1] = myc2d(Gd,Ts);
[zk,dzk] = zpowk(k);
C = convol(C,zk);

// simulation parameters g_s_cl2.xcos ------------
N = 1;
st = 1; // desired change in setpoint
st1 = 0; // magnitude of disturbance
t_init = 0; // simulation start time
t_final = 1.5; // simulation end time

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Np,Rcp] = cosfil_ip(N,Rc); // N/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Cp,Dp] = cosfil_ip(C,D); // C/D


## Example 9.15: Evaluation_of_continuous_time_controller.sce

In [ ]:
// Evaluation of continuous time controller for the case study presented in Example 9.13 on page 349.
// 9.15

clear
exec('tf.sci',-1);
exec('myc2d.sci',-1);
exec('zpowk.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polyno.sci',-1);

num = 200;
den = convol([0.05 1],[0.05 1]);
den = convol([10 1],den);
G = tf(num,den); Ts = 0.005; 
[B,A,k] = myc2d(G,Ts);
[zk,dzk] = zpowk(k); //int = 0;

// Sigurd's feedback controller'
numb = 0.5*convol([1 2],[0.05 1]);
denb = convol([1 0],[0.005 1]);
Gb = tf(numb,denb);
[Sb,Rb,kb] = myc2d(Gb,Ts);
[zkb,dzkb] = zpowk(kb);
Sb = convol(Sb,zkb);

// Sigurd's feed forward controller'
numf = [0.5 1];
denf = convol([0.65 1],[0.03 1]);
Gf = tf(numf,denf);
[Sf,Rf,kf] = myc2d(Gf,Ts);
[zkf,dzkf] = zpowk(kf);
Sf = convol(Sf,zkf);

// Margins
simp_mode(%f);
L = G*Gb;
Gm = g_margin(L); // ------
Pm = p_margin(L); // ------
Lnum = convol(Sb,convol(zk,B));
Lden = convol(Rb,A);
L = tf(Lnum,Lden,Ts);
DGm = g_margin(L); // ------
DPm = p_margin(L); // ------

// Noise
num1 = 100; den1 = [10 1];

// simulation parameters for 
// entirely continuous simulation: g_s_cl3.xcos
// hybrid simulation: g_s_cl6.xcos
st = 1; // desired change in setpoint
st1 = 0;
t_init = 0; // simulation start time
t_final = 5; // simulation end time

num = polyno(num,'s'); den = polyno(den,'s');
Numb = polyno(numb,'s'); Denb = polyno(denb,'s');
Numf = polyno(numf,'s'); Denf = polyno(denf,'s'); 
Num1 = polyno(num1,'s'); Den1 = polyno(den1,'s'); 

[Sbp,Rbp] = cosfil_ip(Sb,Rb);
[Sfp,Rfp] = cosfil_ip(Sf,Rf);



## Example 9.16: System_type_with_2_DOF_controller.sce

In [ ]:
// System type with 2-DOF controller. It is used to arrive at the results Example 9.14.
// 9.16

exec('polsplit3.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('pp_im.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('seshft.sci',-1);
exec('zpowk.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polyno.sci',-1);

B = 1; A = [1 -1]; k = 1; zk = zpowk(k); Ts = 1; 
phi = [1 -0.5];

Delta = 1; // Choice of internal model of step
[Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,Delta);

// simulation parameters for stb_disc.xcos
st = 1; // desired step change
t_init = 0; // simulation start time
t_final = 20; // simulation end time
xInitial = [0 0];
C = 0; D = 1; N_var = 0;

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Rcp1,Rcp2] = cosfil_ip(1,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Bp,Ap] = cosfil_ip(B,A); // B/A
[zkp1,zkp2] = cosfil_ip(zk,1); // zk/1
[Cp,Dp] = cosfil_ip(C,D); // C/D


## Example 9.17: Illustrating_the_benefit_of_cancellation.sce

In [ ]:
// Illustrating the benefit of cancellation. It is used to arrive at the results of Example 9.15.
// 9.17

exec('pp_im.sci',-1);
exec('pp_pid.sci',-1);
exec('zpowk.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polyno.sci',-1);
exec('cosfil_ip.sci',-1);


// test problem to demonstrate benefits of 2_dof
// Ts = 1; B = [1 0.9]; A = conv([1 -1],[1 -0.8]); k = 1;
Ts = 1; k = 1;
B = convol([1 0.9],[1 -0.8]); A = convol([1 -1],[1 -0.5]);
 
// closed loop characteristic polynomial
phi = [1 -1 0.5];

Delta = 1; // Choice of internal model of step
control = 1;
if control == 1, // 1-DOF with no cancellation
   [Rc,Sc] = pp_pid(B,A,k,phi,Delta);
   Tc = Sc; gamm = 1;
else // 2-DOF
   [Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,Delta);
end

// simulation parameters for stb_disc.mdl
[zk,dzk] = zpowk(k);
st = 1; // desired step change
t_init = 0; // simulation start time
t_final = 20; // simulation end time
xInitial = [0 0];
C = 0; D = 1; N_var = 0;

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Rcp1,Rcp2] = cosfil_ip(1,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Bp,Ap] = cosfil_ip(B,A); // B/A
[zkp1,zkp2] = cosfil_ip(zk,1); // zk/1
[Cp,Dp] = cosfil_ip(C,D); // C/D





## Example 9.18: Anti_windup_control_of_IBM_Lotus_Domino_server.sce

In [ ]:
// Anti windup control (AWC) of IBM Lotus Domino server, studied in Example 9.16 on page 357. It can be used for the follwoing situations: with and without saturation, and with and without AWC.
// 9.18

exec('pp_im2.sci',-1);
exec('desired.sci',-1);
exec('zpowk.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polsplit3.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('polyno.sci',-1);
exec('cindep.sci',-1);
exec('poladd.sci',-1);

//  Transfer function
B = 0.47; A = [1 -0.43]; k = 1; 
[zk,dzk] = zpowk(k);
 
// Transient specifications
rise = 10; epsilon = 0.01; Ts = 1;
phi = desired(Ts,rise,epsilon);

// Controller design
delta = [1 -1]; // internal model of step used
[Rc,Sc,Tc,gamm,F] = pp_im2(B,A,k,phi,delta);

// Study of Antiwindup Controller

key = x_choose(['Simulate without any saturation limits'; 
         'Simulate saturation, but do not use AWC'; 
         'Simulate saturation with AWC in place'; 
         'Simulate with AWC, without saturation limits'],...
         ['Please choose one of the following']);

if key ==0
  disp('Invalid choice');
  return;
elseif key == 1
  U = 2; L = -2; P = 1; F = Rc; E = 0; PSc = Sc; PTc = Tc;
elseif key == 2
  U = 1; L = -1; P = 1; F = Rc; E = 0; PSc = Sc; PTc = Tc;
else
  if key == 3 // Antiwindup controller and with saturation
     U = 1; L = -1; 
  elseif key == 4 // Antiwindup controller, but no saturation
     U = 2; L = -2;
  end
  P = A;
  dF = length(F) - 1;
  PRc = convol(P,Rc); dPRc = length(PRc) - 1;
  [E,dE] = poladd(F,dF,-PRc,dPRc);
  PSc = convol(P,Sc); PTc = convol(P,Tc);
end

// Setting up simulation parameters for stb_disc_sat
t_init = 0; // first step begins
st = 1; // height of first step
t_init2 = 500; // second step begins
st2 = -2; // height of second step
t_final = 1000; // simulation end time
st1 = 0; // no disturbance input
C = 0; D = 1; N_var = 0;

[PTcp1,PTcp2] = cosfil_ip(PTc,1); // PTc/1
[Fp1,Fp2] = cosfil_ip(1,F); // 1/F
[Ep,Fp] = cosfil_ip(E,F); // E/F
[PScp1,PScp2] = cosfil_ip(PSc,1); // PSc/1
[Bp,Ap] = cosfil_ip(B,A); // B/A
[zkp1,zkp2] = cosfil_ip(zk,1); // zk/1
[Cp,Dp] = cosfil_ip(C,D); // C/D





## Example 9.19: Demonstration_of_usefulness_of_negative_PID_parameters.sce

In [ ]:
// Demonstration of usefulness of negative PID parameters, discussed in Example 9.17 on page 361.
// 9.19

exec('iodelay.sci',-1);
exec('delc2d.sci',-1);
exec('desired.sci',-1);
exec('pp_pid.sci',-1);
exec('cosfil_ip.sci',-1);
exec('tf.sci',-1);
exec('flip.sci',-1);
exec('zpowk.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);

// Discretize the continuous plant
num = 1; den = [2 1]; tau = 0.5;
G1 = tf(num,den);
G = iodelay(G1,tau);
Ts = 0.5;
[B,A,k] = delc2d(G,G1,Ts);

// Specify transient requirements
epsilon = 0.05; rise = 5;
phi = desired(Ts,rise,epsilon);

// Design the controller
Delta = [1 -1];
[Rc,Sc] = pp_pid(B,A,k,phi,Delta);

// parameters for simulation using g_s_cl
Tc = Sc; gamm = 1; N = 1; 
C = 0; D = 1; N_var = 0; 
st = 1; t_init = 0; t_final = 20;

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Np,Rcp] = cosfil_ip(N,Rc); // N/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Cp,Dp] = cosfil_ip(C,D); // C/D
Num = numer(G1);
Den = denom(G1);


## Example 9.1: Pole_placement_controller_for_magnetically_suspended_ball_problem.sce

In [ ]:
// Pole placement controller for magnetically suspended ball problem, discussed in Example 9.3 on page 331.
// 9.1

exec('myc2d.sci',-1);
exec('desired.sci',-1);
exec('zpowk.sci',-1);
exec('polsplit2.sci',-1);
exec('polsize.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polmul.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('xdync.sci',-1);
exec('left_prm.sci',-1);
exec('rowjoin.sci',-1);
exec('pp_basic.sci',-1);
exec('polyno.sci',-1);
exec('cosfil_ip.sci',-1);

// Magnetically suspended ball problem
// Operating conditions
M = 0.05; L = 0.01; R = 1; K = 0.0001; g = 9.81;

//Equilibrium conditions
hs = 0.01; is = sqrt(M*g*hs/K);

// State space matrices
a21 = K*is^2/M/hs^2; a23 = - 2*K*is/M/hs; a33 = - R/L;
b3 = 1/L;
a1 = [0 1 0; a21 0 a23; 0 0 a33];
b1 = [0; 0; b3]; c1 = [1 0 0]; d1 = 0;

// Transfer functions
G = syslin('c',a1,b1,c1,d1); Ts = 0.01; 
[B,A,k] = myc2d(G,Ts);

//polynomials are returned
[Ds,num,den] = ss2tf(G);
num = clean(num); den = clean(den);

// Transient specifications
rise = 0.15; epsilon = 0.05;
phi = desired(Ts,rise,epsilon);

// Controller design
[Rc,Sc,Tc,gamm] = pp_basic(B,A,k,phi);

// Setting up simulation parameters for basic.xcos
st = 0.0001; // desired change in h, in m.
t_init = 0; // simulation start time
t_final = 0.5; // simulation end time

// Setting up simulation parameters for c_ss_cl.xcos
N_var = 0; xInitial = [0 0 0]; N = 1; C = 0; D = 1;

[Tc1,Rc1] = cosfil_ip(Tc,Rc); // Tc/Rc 
[Sc2,Rc2] = cosfil_ip(Sc,Rc); // Sc/Rc

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Np,Rcp] = cosfil_ip(N,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Cp,Dp] = cosfil_ip(C,D); // C/D

## Example 9.20: PID_controller_design.sci

In [ ]:
// Solution to Aryabhatta's identity arising in PID controller design, namely Eq. 9.37 on page 363.
// 9.20
 
function [Rc,Sc] = pp_pid(B,A,k,phi,Delta)

// Setting up and solving Aryabhatta identity
dB = length(B) - 1; dA = length(A) - 1;
[zk,dzk] = zpowk(k);
[N,dN] = polmul(B,dB,zk,dzk);
dDelta = length(Delta)-1;
[D,dD] = polmul(A,dA,Delta,dDelta);
dphi = length(phi)-1;
[Sc,dSc,R,dR] = xdync(N,dN,D,dD,phi,dphi);
Rc = convol(R,Delta);
endfunction;

## Example 9.21: DC_motor_with_PID_control_tuned_through_pole_placement_technique.sce

In [ ]:
// DC motor with PID control, tuned through pole placement technique, as in Example 9.18.
// 9.21

exec('desired.sci',-1);
exec('pp_pid.sci',-1);
exec('cosfil_ip.sci',-1);
exec('pd.sci',-1);
exec('polyno.sci',-1);
exec('myc2d.sci',-1);
exec('zpowk.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);

// Motor control problem
// Transfer function

a = [-1 0; 1 0]; b = [1; 0]; c = [0 1]; d = 0;
G = syslin('c',a,b,c,d); Ts = 0.25;
[B,A,k] = myc2d(G,Ts);
[Ds,num,den] = ss2tf(G);

// Transient specifications
rise = 3; epsilon = 0.05;
phi = desired(Ts,rise,epsilon);

// Controller design
Delta = 1;  //No internal model of step used
[Rc,Sc] = pp_pid(B,A,k,phi,Delta);

// continuous time controller
[K,taud,N] = pd(Rc,Sc,Ts);
numb = K*[1 taud*(1+1/N)]; denb = [1 taud/N];
numf = 1; denf = 1;

// simulation parameters
st = 1;  // desired change in position
t_init = 0; // simulation start time
t_final = 20; // simulation end time
st1 = 0;

// continuous controller simulation: g_s_cl3.xcos
num1 = 0; den1 = 1; 

// discrete controller simulation: g_s_cl2.xcos
// u1: -0.1 to 0.8
// y1: 0 to 1.4
C = 0; D = 1; N = 1; gamm = 1; Tc = Sc; 

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Np,Rcp] = cosfil_ip(N,Rc); // N/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Cp,Dp] = cosfil_ip(C,D); // C/D
Numb = polyno(numb,'s');
Denb = polyno(denb,'s');
Numf = polyno(numf,'s');
Denf = polyno(denf,'s');
Num1 = polyno(num1,'s');
Den1 = polyno(den1,'s');

## Example 9.22: PD_control_law_from_polynomial_coefficients.sci

In [ ]:
// PD control law from polynomial coefficients, as explained in Sec. 9.8.
// 9.22

function [K,taud,N] = pd(Rc,Sc,Ts)

// Both Rc and Sc have to be degree one polynomials

s0 = Sc(1); s1 = Sc(2);
r1 = Rc(2);
K = (s0+s1)/(1+r1);
N = (s1-s0*r1)/r1/(s0+s1);
taudbyN = -Ts*r1/(1+r1);
taud = taudbyN * N;
endfunction;

## Example 9.2: Discretization_of_continuous_transfer_function.sci

In [ ]:
// Discretization of continuous transfer function. The result is numerator and denominator in powers of z^{-1} and the delay term k.
// 9.2
// function [B,A,k] = myc2d(G,Ts)
// Produces numerator and denominator of discrete transfer 
// function in powers of z^{-1}
// G is continuous transfer function; time delays are not allowed
// Ts is the sampling time, all in consistent time units

function [B,A,k] = myc2d(G,Ts)
H = ss2tf(dscr(G,Ts));
num1 = coeff(H('num'));
den1 = coeff(H('den'));//-------------
A = den1(length(den1):-1:1);
num2 = num1(length(num1):-1:1);  //flip
nonzero = find(num1); 
first_nz = nonzero(1);
B = num2(first_nz:length(num2)); //-------------
k = length(den1) - length(num1);
endfunction


## Example 9.3: Procedure_to_split_a_polynomial_into_good_and_bad_factors.sci

In [ ]:
// Procedure to split a polynomial into good and bad factors, as discussed in Sec. 9.2.
// 9.3
// function [goodpoly,badpoly] = polsplit2(fac,a)
// Splits a scalar polynomial of z^{-1} into good and bad 
// factors. 
// Input is a polynomial in increasing degree of z^{-1}
// Optional input is a, where a <= 1.
// Factor that has roots of z^{-1} outside a is called
// good and the rest bad.
// If a is not specified, it will be assumed as 1-1.0e-5

function [goodpoly,badpoly] = polsplit2(fac,a)
if argn(2) == 1, a = 1-1.0e-5; end  
if a>1 error('good polynomial is unstable'); end
fac1 = poly(fac(length(fac):-1:1),'z','coeff');
rts1 = roots(fac1);
rts = rts1(length(rts1):-1:1);

// extract good and bad roots
badindex = find(abs(rts)>=a); // mtlb_find has been replaced by find
badpoly = coeff(poly((rts(badindex)),'z','roots'));
goodindex = find(abs(rts)<a); // mtlb_find has been replaced by find
goodpoly = coeff(poly(rts(goodindex),'z','roots'));

// scale by equating the largest terms
[m,index] = max(abs(fac));
goodbad = convol(goodpoly,badpoly);
goodbad1 = goodbad(length(goodbad):-1:1);//--
factor1 = fac(index)/goodbad1(index);//--
goodpoly = goodpoly * factor1;
goodpoly = goodpoly(length(goodpoly):-1:1);
badpoly = badpoly(length(badpoly):-1:1);
endfunction;




## Example 9.4: Calculation_of_desired_closed_loop_characteristic_polynomial.sci

In [ ]:
// Calculation of desired closed loop characteristic polynomial, as discussed in Sec. 7.7.
// 9.4

// function [phi,dphi] = desired(Ts,rise,epsilon)
// Based on transient requirements, 
// calculates closed loop characteristic polynomial

function [phi,dphi] = desired(Ts,rise,epsilon)
Nr = rise/Ts; omega = %pi/2/Nr; rho = epsilon^(omega/%pi); 
phi = [1 -2*rho*cos(omega) rho^2]; dphi = length(phi)-1;
endfunction;

## Example 9.5: Design_of_2_DOF_pole_placement_controller.sci

In [ ]:
// Design of 2-DOF pole placement controller, as discussed in Sec. 9.2.
// 9.5

// function [Rc,Sc,Tc,gamma] = pp_basic(B,A,k,phi)
// calculates pole placement controller


function [Rc,Sc,Tc,gamm] = pp_basic(B,A,k,phi)

// Setting up and solving Aryabhatta identity
[Ag,Ab] = polsplit2(A); dAb = length(Ab) - 1;
[Bg,Bb] = polsplit2(B); dBb = length(Bb) - 1;

[zk,dzk] = zpowk(k);

[N,dN] = polmul(Bb,dBb,zk,dzk);
dphi = length(phi) - 1;

[S1,dS1,R1,dR1] = xdync(N,dN,Ab,dAb,phi,dphi);

// Determination of control law
Rc = convol(Bg,R1); Sc = convol(Ag,S1);
Tc = Ag; gamm = sum(phi)/sum(Bb); 

endfunction;


## Example 9.6: Evaluates_z_to_the_power_k.sci

In [ ]:
// Evaluates z^-k.
// 9.6

function [zk,dzk] = zpowk(k)
zk = zeros(1,k+1); zk(1,k+1) = 1;
dzk = k;
endfunction

## Example 9.7: Simulation_of_closed_loop_system_with_an_unstable_controller.sce

In [ ]:
// Simulation of closed loop system with an unstable controller, as discussed in Example 9.5 on page 335.
// 9.7

exec('desired.sci',-1);
exec('zpowk.sci',-1);
exec('polmul.sci',-1);
exec('polsplit2.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('seshft.sci',-1);
exec('cosfil_ip.sci',-1);
exec('pp_basic.sci',-1);

Ts = 1; B = [1 -3]; A = [1 2 -8]; k = 1;
// Since k=1, tf is of the form z^-1
[zk,dzk] = zpowk(k); // int1 = 0;//---- int1

// Transient specifications
rise = 10; epsilon = 0.1;
phi = desired(Ts,rise,epsilon);

// Controller design
[Rc,Sc,Tc,gamm] = pp_basic(B,A,k,phi);

// simulation parameters for basic_disc.xcos
//While simulating for t_final = 100, set the limit of Y axis of each scope
//u1: -0.2 to 3
//y1: -0.1 to 1.2
st = 1.0; // Desired change in setpoint
t_init = 0; // Simulation start time
t_final = 1000; // Simulation end time

// Simulation parameters for stb_disc.xcos
N_var = 0; C = 0; D = 1; N = 1; 

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Rcp1,Rcp2] = cosfil_ip(1,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Bp,Ap] = cosfil_ip(B,A); // B/A
[zkp1,zkp2] = cosfil_ip(zk,1); // zk/1
[Cp,Dp] = cosfil_ip(C,D); // C/D

[Tcp,Rcp] = cosfil_ip(Tc,Rc); // Tc/Rc
[Scp_b,Rcp_b] = cosfil_ip(Sc,Rc); // Sc/Rc




## Example 9.8: Pole_placement_controller_using_internal_model_principle.sci

In [ ]:
// Pole placement controller using internal model principle, as discussed in Sec. 9.4.
// 9.8

// function [Rc,Sc,Tc,gamma,phit] = pp_im(B,A,k,phi,Delta)
// Calculates 2-DOF pole placement controller.  

function [Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,Delta)

// Setting up and solving Aryabhatta identity
[Ag,Ab] = polsplit3(A); dAb = length(Ab) - 1;
[Bg,Bb] = polsplit3(B); dBb = length(Bb) - 1;

[zk,dzk] = zpowk(k);

[N,dN] = polmul(Bb,dBb,zk,dzk);
dDelta = length(Delta)-1;
[D,dD] = polmul(Ab,dAb,Delta,dDelta);
dphi = length(phi)-1;

[S1,dS1,R1,dR1] = xdync(N,dN,D,dD,phi,dphi);

// Determination of control law
Rc = convol(Bg,convol(R1,Delta)); Sc = convol(Ag,S1);
Tc = Ag; gamm = sum(phi)/sum(Bb); 
endfunction;

## Example 9.9: EX9_9.sce

In [ ]:
// Pole placement controller, with internal model of a step, for the magnetically suspended ball problem, as discussed in Example 9.8 on page 339.
// 9.9

// PP control with internal model for ball problem
exec('desired.sci',-1);
exec('pp_im.sci',-1);
exec('myc2d.sci',-1);
exec('polsplit3.sci',-1);
exec('zpowk.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('cindep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polyno.sci',-1);

// Operating conditions
M = 0.05; L = 0.01; R = 1; K = 0.0001; g = 9.81;

// Equilibrium conditions
hs = 0.01; is = sqrt(M*g*hs/K);

// State space matrices
a21 = K*is^2/M/hs^2; a23 = - 2*K*is/M/hs; a33 = - R/L;
b3 = 1/L;
a1 = [0 1 0; a21 0 a23; 0 0 a33];
b1 = [0; 0; b3]; c1 = [1 0 0]; d1 = 0;

// Transfer functions
G = syslin('c',a1,b1,c1,d1); Ts = 0.01; [B,A,k] = myc2d(G,Ts);

// Transient specifications 
rise = 0.1; epsilon = 0.05;
phi = desired(Ts,rise,epsilon);

// Controller design
Delta = [1 -1];  //internal model of step used
[Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,Delta);

// simulation parameters for c_ss_cl.xcos
st = 0.0001;  //desired change in h, in m.
t_init = 0; // simulation start time
t_final = 0.5; //simulation end time
xInitial = [0 0 0];
N = 1; C = 0; D = 1; N_var = 0;

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Np,Rcp] = cosfil_ip(N,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Cp,Dp] = cosfil_ip(C,D); // C/D
